In [ ]:
from pymongo import MongoClient

import os
from dotenv import load_dotenv
load_dotenv()


In [38]:
db_constants = {'connection_string': os.getenv('MONGODB_CONNECTION_STRING'),
				'embeddings_db': 'embeddings',
				'embeddings_collection': 'clusters',
				'histories_db': 'chat_histories',
				'user_id': '0000'}

example_history = [
{"role": "user", "content": "What is the capital of France?"},
{"role": "chatbot", "content": "The capital of France is Paris."},
{"role": "user", "content": "And what is it's population?"},
]

In [39]:
message = {'Rule': 'User',
           'Content': 'Hiiii'}


In [40]:
def get_chat_history(db_constants, user_id):
    client = MongoClient(db_constants['connection_string'])
    collection = client[db_constants['histories_db']][db_constants[user_id]]
    messages = collection.find()
    # Format the messages in the desired 'Rule: Content' format
    formatted_messages = [f"{message['Rule']}: {message['Content']}" for message in messages]
    return formatted_messages


def update_chat_history(db_constants, user_id, message):
    client = MongoClient(db_constants['connection_string'])
    collection = client[db_constants['histories_db']][db_constants[user_id]]
    collection.insert_one(message)


def reset_chat_history(db_constants, user_id):
    client = MongoClient(db_constants['connection_string'])
    collection = client[db_constants['histories_db']][db_constants[user_id]]
    collection.delete_many({})



In [46]:
for message in example_history:
    update_chat_history(db_constants, '0000', message)
    
    

KeyError: '0000'

In [45]:
client = MongoClient(db_constants['connection_string'])
collection = client[db_constants['histories_db']]['0000']
collection.insert_one(example_history[0])

InsertOneResult(ObjectId('6702b92d137caee9f6cc8484'), acknowledged=True)